In [1]:
import ray
ray.init()

2023-07-31 15:41:43,068	INFO worker.py:1612 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Python version:,3.9.17
Ray version:,2.6.1
Dashboard:,http://127.0.0.1:8265


In [2]:
resources = ray.cluster_resources()
print(resources)

{'CPU': 32.0, 'node:__internal_head__': 1.0, 'memory': 68581817754.0, 'GPU': 1.0, 'accelerator_type:G': 1.0, 'object_store_memory': 33677921894.0, 'node:158.125.237.153': 1.0}


In [3]:
#env = gym.make('FrankaPickPlaceFixed-v0')

In [4]:
def env_creator(env_config={}):
    import gymnasium as gym
    env = gym.make('FetchPickAndPlace-v2', render_mode='human', reward_type='dense', max_episode_steps=100)
    #env = CustomWrapper(env)
    env.reset()
    return env

In [5]:
from ray.tune.registry import register_env
register_env("Gymnasium_Pick_Place_0_Dense", env_creator)

In [6]:
# Serialize with cloudpickle and save to a file
#with open("Gymnasium_Pick_Place_0.pkl", "wb") as f:
   # cloudpickle.dump(env_creator, f)

In [7]:
from ray import tune

In [ ]:
tune.run("PPO",
             # algorithm specific configuration
             config={"env": "Gymnasium_Pick_Place_0_Dense",  #
                     "framework": "torch",
                     #"seed":958,
                     "num_gpus": 1,
                     "num_rollout_workers": 24,
                     #"num_envs_per_worker": 1,
                     #"num_gpus_per_worker": 0,
                     "train_batch_size": 512,
                     "use_critic":True,
                     "use_gae": True,
                     "lambda": 0.99,
                     "sgd_minibatch_size": 32,
                     "num_sgd_iter": 8,
                     "shuffle_sequences": False,
                     "entropy_coeff": 0.00015,
                     "clip_param": 0.1,
                     "grad_clip": 5,
                     "lr": 0.00015,
                     #"num_cpus_per_worker": 1,
                     #"model": {"custom_model": "pa_model", },
                     "evaluation_interval": 1,
                     "evaluation_num_episodes": 2,
                     "evaluation_parallel_to_training": True,
                     "evaluation_num_workers": 1
                     },
             local_dir="../R3L-LOGS",  # directory to save results
             checkpoint_freq=2,  # frequency between checkpoints
             keep_checkpoints_num=6,
             stop={"training_iteration": 2_000_000}
             )

2023-07-31 15:41:54,381	INFO tune.py:657 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949
/home/lunet/cocp5/anaconda3/envs/R3L/lib/python3.9/site-packages/ray/tune/tune.py:258: UserWarning: Passing a `local_dir` is deprecated and will be removed in the future. Pass `storage_path` instead or set the `RAY_AIR_LOCAL_CACHE_DIR` environment variable instead.
  warnings.warn(
2023-07-31 15:41:55,485	WARNING deprecation.py:50 -- DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
2023-07-31 15:41:55,531	WARNING deprecation.py:50 -- DeprecationWarning: `build_tf_policy` has been deprecated. This will raise an error in the future!
2023-07-31 15:41:55,533	WARNING deprecation.py:50 -- DeprecationWarning: `build_policy_class` has been deprecated. This will raise an error in the future!
2023-0

2023-07-31 15:41:55,590	WARNING algorithm_config.py:2534 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
2023-07-31 15:41:55,590	WARNING deprecation.py:50 -- DeprecationWarning: `AlgorithmConfig.evaluation(evaluation_num_episodes=..)` has been deprecated. Use `AlgorithmConfig.evaluation(evaluation_duration=.., evaluation_duration_unit='episodes')` instead. This will raise an error in the future!
(pid=245082) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(PPO pid=245082) 2023-07-31 15:41:57,776	WARNING algorithm_config.py:2534 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`

Trial name,agent_timesteps_total,connector_metrics,counters,custom_metrics,episode_len_mean,episode_media,episode_reward_max,episode_reward_mean,episode_reward_min,episodes_this_iter,evaluation,info,num_agent_steps_sampled,num_agent_steps_trained,num_env_steps_sampled,num_env_steps_sampled_this_iter,num_env_steps_sampled_throughput_per_sec,num_env_steps_trained,num_env_steps_trained_this_iter,num_env_steps_trained_throughput_per_sec,num_faulty_episodes,num_healthy_workers,num_in_flight_async_reqs,num_remote_worker_restarts,num_steps_trained_this_iter,perf,policy_reward_max,policy_reward_mean,policy_reward_min,sampler_perf,sampler_results,timers
PPO_Gymnasium_Pick_Place_0_Dense_65540_00000,907264,"{'ObsPreprocessorConnector_ms': 0.029889345169067383, 'StateBufferConnector_ms': 0.005986213684082031, 'ViewRequirementAgentConnector_ms': 0.2951641082763672}","{'num_env_steps_sampled': 907264, 'num_env_steps_trained': 0, 'num_agent_steps_sampled': 907264, 'num_agent_steps_trained': 0}",{},100,{},-4.10593,-27.0151,-53.7582,16,"{'sampler_results': {'episode_reward_max': -14.111181650510863, 'episode_reward_min': -14.335487667624562, 'episode_reward_mean': -14.223334659067714, 'episode_len_mean': 100.0, 'episode_media': {}, 'episodes_this_iter': 2, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [-14.335487667624562, -14.111181650510863], 'episode_lengths': [100, 100]}, 'sampler_perf': {'mean_raw_obs_processing_ms': 0.5822462878714739, 'mean_inference_ms': 0.9877099713973186, 'mean_action_processing_ms': 0.14377741186251913, 'mean_env_wait_ms': 24.66895810428265, 'mean_env_render_ms': 0.0}, 'num_faulty_episodes': 0, 'connector_metrics': {'ObsPreprocessorConnector_ms': 0.016677379608154297, 'StateBufferConnector_ms': 0.0034570693969726562, 'ViewRequirementAgentConnector_ms': 0.1881718635559082}}, 'episode_reward_max': -14.111181650510863, 'episode_reward_min': -14.335487667624562, 'episode_reward_mean': -14.223334659067714, 'episode_len_mean': 100.0, 'episode_media': {}, 'episodes_this_iter': 2, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [-14.335487667624562, -14.111181650510863], 'episode_lengths': [100, 100]}, 'sampler_perf': {'mean_raw_obs_processing_ms': 0.5822462878714739, 'mean_inference_ms': 0.9877099713973186, 'mean_action_processing_ms': 0.14377741186251913, 'mean_env_wait_ms': 24.66895810428265, 'mean_env_render_ms': 0.0}, 'num_faulty_episodes': 0, 'connector_metrics': {'ObsPreprocessorConnector_ms': 0.016677379608154297, 'StateBufferConnector_ms': 0.0034570693969726562, 'ViewRequirementAgentConnector_ms': 0.1881718635559082}, 'num_agent_steps_sampled_this_iter': 200, 'num_env_steps_sampled_this_iter': 200, 'timesteps_this_iter': 200, 'num_healthy_workers': 1, 'num_in_flight_async_reqs': 0, 'num_remote_worker_restarts': 0}","{'learner': {'__all__': {'num_agent_steps_trained': 32.0, 'num_env_steps_trained': 512.0, 'total_loss': 0.644969266575572}, 'default_policy': {'total_loss': 0.644969266575572, 'policy_loss': -0.008972844127129065, 'vf_loss': 0.6519905774839572, 'vf_loss_unclipped': 2.1903748390250257, 'vf_explained_var': 0.9856102466583252, 'entropy': 9.854192845523357, 'mean_kl_loss': 0.009828502213622414, 'gradients_default_optimizer_global_norm': 43.98967771278694, 'curr_lr': 0.00015, 'curr_entropy_coeff': 0.00015, 'curr_kl_coeff': 0.3489513397216797}}, 'num_env_steps_sampled': 907264, 'num_env_steps_trained': 0, 'num_agent_steps_sampled': 907264, 'num_agent_steps_trained': 0}",907264,0,907264,512,756.546,0,0,0,0,24,0,0,0,"{'cpu_util_percent': 16.1, 'ram_util_percent': 31.060000000000002}",{},{},{},"{'mean_raw_obs_processing_ms': 0.8831361200914634, 'mean_inference_ms': 1.30812145802647, 'mean_action_processing_ms': 0.1785313443692792, 'mean_env_wait_ms': 23.068296180600257, 'mean_env_render_ms': 0.0}","{'episode_reward_max': -4.10593421972185, 'episod

In [ ]:
import gymnasium as gym
env = gym.make('FetchPickAndPlace-v2', render_mode='rgb_array', reward_type='dense', max_episode_steps=100)
#env = CustomWrapper(env)
env.reset()

In [ ]:
import numpy as np

from ray.rllib.policy.policy import Policy     

# Use the `from_checkpoint` utility of the Policy class:
my_restored_policy = Policy.from_checkpoint("../R3L-LOGS/PPO/PPO_Gymnasium_Pick_Place_0_Dense_df6f8_00000_0_2023-07-28_10-19-37/checkpoint_004994/policies/default_policy")

# Use the restored policy for serving actions.
obs,  reward, terminated, truncated, info =   env.step(env.action_space.sample())# individual CartPole observation
#action = my_restored_policy.compute_single_action(np.append(obs.get('observation'), obs.get('achieved_goal'), obs.get('desired_goal')))


In [ ]:
import matplotlib.pyplot as plt
from IPython import display as ipythondisplay
img = plt.imshow(env.render())
for i in range(0, 512):
    
    action = my_restored_policy.compute_single_action(np.concatenate([obs.get('observation'), obs.get('desired_goal'), obs.get('achieved_goal')]))

    obs,  reward, terminated, truncated, info =   env.step(env.action_space.sample())# individual CartPole observation

    #print(f"Computed action {action} from given CartPole observation.")
    img.set_data(env.render()) # Just update the data
    ipythondisplay.display(plt.gcf())
    ipythondisplay.clear_output(wait=True)